In [1]:
import os

os.environ["SEED"] = "0"

import cocoex
import matplotlib.pyplot as plt
import nevergrad as ng
import numpy as np
import scipy

from src.solver.BBOB_POP_Solver import BBOB_POP_Solver
from src.instance.BBOB_Instance import BBOB_Instance

In [2]:
suite = cocoex.Suite("bbob", "", "function_indices:1 dimensions:2 instance_indices:1-15")

In [3]:
suite

Suite(b'bbob', b'', b'function_indices:1 dimensions:2 instance_indices:1-15')

In [4]:
len(list(suite))
# 6 dimensions X 24 functions X 15 instances = 2160 problems

15

In [5]:
instance = BBOB_Instance(
    function_index=1,
    dimension=2,
    instance_index=1,
    cut_off_cost=5.0,
    cut_off_time=0.5,
)
instance

BBOB_Instance(id=function_indices:1 dimensions:2 instance_indices:1)

In [6]:
solver = BBOB_POP_Solver()
solver.config

Configuration(values={
  'ALGORITHM': 'PSO',
  'CMA_ELITIST': True,
  'CMA_POPSIZE': 98,
  'CMA_POPSIZE_FACTOR': 3.1155796790116,
  'CMA_RANDOM_INIT': False,
  'CMA_SCALE': 0.5070172631967,
  'DE_CROSSOVER': 0.6431036240952,
  'DE_F1': 0.2482706500629,
  'DE_F2': 0.2893054427676,
  'DE_INITIALIZATION': 'SO',
  'DE_POPSIZE': 11,
  'DE_RECOMMENDATION': 'mean',
  'DE_SCALE': 0.3449599655442,
  'PSO_OMEGA': 0.9895364542533,
  'PSO_PHIG': 1.0065832050796,
  'PSO_PHIP': 1.8126591789305,
  'PSO_POPSIZE': 10,
})

In [7]:
solver.solve(
    instance=instance,
    prefix="test",
    calculate_features=False,
    cache=False
)

[2025-07-10 07:00:49] DEBUG     solve(prefix=test, solver=Solver(id=972306317029875343), instance=BBOB_Instance(id=function_indices:1 dimensions:2 instance_indices:1))


AttributeError: 'BBOB_Instance' object has no attribute 'dimension'

In [ ]:
print(solver.config)

instance.reset_problem()
kwargs = dict(solver.config)
algorithm = solver.config["ALGORITHM"]

def _format_key(key: str, algorithm) -> str:
    key = key[len(algorithm) + 1:]
    if not (algorithm == "DE" and key in ["F1", "F2"]):
        key = key.lower()
    return key

kwargs = {_format_key(k, algorithm): v for k, v in kwargs.items() if k.startswith(algorithm)}

if algorithm == "PSO":
    optimizer_class = ng.families.ConfPSO(**kwargs)
elif algorithm == "DE":
    optimizer_class = ng.families.DifferentialEvolution(**kwargs)
elif algorithm == "CMA":
    optimizer_class = ng.families.ParametrizedCMA(**kwargs)
else:
    raise ValueError(f"Unknown algorithm: {algorithm}")

optimizer = optimizer_class(parametrization=instance.dimension, budget=1e6)
recommendation = optimizer.minimize(instance.problem, max_time=instance.cut_off_time)
recommendation.value, instance.problem.evaluations, instance.problem.final_target_hit

Configuration(values={
  'ALGORITHM': 'DE',
  'CMA_ELITIST': True,
  'CMA_POPSIZE': 27,
  'CMA_POPSIZE_FACTOR': 1.9924489939203,
  'CMA_RANDOM_INIT': True,
  'CMA_SCALE': 0.6930995888842,
  'DE_CROSSOVER': 0.7519767004477,
  'DE_F1': 0.3459925631343,
  'DE_F2': 0.5720256177582,
  'DE_INITIALIZATION': 'parametrization',
  'DE_POPSIZE': 39,
  'DE_RECOMMENDATION': 'pessimistic',
  'DE_SCALE': 0.7562708519322,
  'PSO_OMEGA': 0.3707155809651,
  'PSO_PHIG': 2.4297133297554,
  'PSO_PHIP': 0.6268024897845,
  'PSO_POPSIZE': 38,
})


(array([ 0.25280177, -1.15679961]), 892, True)

In [7]:
problem = suite[0]

In [36]:
problem.dimension

2

In [8]:
problem.evaluations

0

In [9]:
problem.final_target_hit

False

In [ ]:
pso_config = ng.families.ConfPSO(
    popsize=20,           # Population size
    omega=0.729,          # Inertia weight
    phip=1.49618,         # Cognitive parameter
    phig=1.49618,         # Social parameter
)

optimizer = pso_config(2, budget=1000)

recommendation = optimizer.minimize(problem)
recommendation.value

array([ 0.25222357, -1.15669464])

In [11]:
scipy.optimize.fmin(problem, problem.initial_solution, disp=True)

Optimization terminated successfully.
         Current function value: 79.480000
         Iterations: 65
         Function evaluations: 124


array([ 0.25281244, -1.15682905])

In [12]:
problem.evaluations

1124

In [13]:
problem.final_target_hit

True

In [24]:
problem = suite[0]

pso_config = ng.families.ConfPSO(
    popsize=20,           # Population size
    omega=0.729,          # Inertia weight
    phip=1.49618,         # Cognitive parameter
    phig=1.49618,         # Social parameter
)

optimizer = pso_config(2, budget=100)

recommendation = optimizer.minimize(problem, max_time=100)
recommendation.value, problem.evaluations, problem.final_target_hit

(array([ 0.15684738, -1.09546823]), 100, False)

In [34]:
problem = suite[0]

de_config = ng.families.DifferentialEvolution(
    popsize=20,
    crossover=0.8,
)

optimizer = de_config(2, budget=10000)

recommendation = optimizer.minimize(problem, max_time=1)
recommendation.value, problem.evaluations, problem.final_target_hit

(array([ 0.25279718, -1.15679968]), 2851, True)

In [14]:
suite = cocoex.Suite("bbob", "", "function_indices:1 dimensions:2 instance_indices:1-15")

for problem in suite:
    xopt = fmin(problem, problem.initial_solution, disp=True)
    print(problem.final_target_hit)

Optimization terminated successfully.
         Current function value: 79.480000
         Iterations: 65
         Function evaluations: 124
True
Optimization terminated successfully.
         Current function value: 394.480000
         Iterations: 71
         Function evaluations: 138
True
Optimization terminated successfully.
         Current function value: -247.110000
         Iterations: 71
         Function evaluations: 139
True
Optimization terminated successfully.
         Current function value: -152.040000
         Iterations: 72
         Function evaluations: 141
True
Optimization terminated successfully.
         Current function value: -25.250000
         Iterations: 58
         Function evaluations: 114
True
Optimization terminated successfully.
         Current function value: 183.010000
         Iterations: 78
         Function evaluations: 149
True
Optimization terminated successfully.
         Current function value: 183.520000
         Iterations: 71
         Function